# miTranscriptome evidence

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4417758/


In [ ]:
import os
import pandas as pd
import glob
import numpy as np

GENERAL = "data"

projects = [f.name for f in os.scandir("data") if f.is_dir()]
bash_projects = " ".join(projects)

for proj in projects:
    try:
        os.makedirs(os.path.join("data",proj,"analysis/miTranscriptome"))
    else:
        print("%s already contains a miTranscriptome directory" %(proj))
###if projects names are not the GEO ids, we suggest to change them. Keys correspond to project names as they are in the folder and values the GEO id
#dict_projects = {'liver_adjacent_totalRNA':'GSE101432', 'hcc_normal_totalRNA':'GSE77314','zou_hcc_RP_totalRNA':'GSE112705', 'GSE193567':'GSE193567'}
#GEO = dict_projects.values()
#GEO_list = list(GEO)

In [ ]:
%%bash -s "$bash_projects"
mitranscriptome=miTranscriptome/mitranscriptome.v2.gtf

module load BEDTools

for proj in $1; do
    gffcompare=data/$proj/analysis/gffcompare/gffcmp.MODIFids.combined.gtf
    
    bedtools intersect -v -a $gffcompare -b $mitranscriptome > data/$proj/analysis/mitranscriptome/nooverlap_mitranscriptome.gtf
    bedtools intersect -wa -wb -a $gffcompare -b $mitranscriptome > data/$proj/analysis/mitranscriptome/overlap_mitranscriptome.gtf
done

In [ ]:
%%bash -s "$bash_projects"

for proj in $1; do
    echo "for project "$proj":"
    
    echo "novel tumor-specific transcripts"
    cat data/$proj/analysis/gffcompare/gffcmp.MODIFids.combined.gtf | cut -d";" -f2 | sort | uniq | wc -l

    echo "novel tumor-specific transcripts evidenced in miTranscriptome"
    cat data/$proj/analysis/mitranscriptome/overlap_mitranscriptome.gtf | cut -d";" -f2 | sort | uniq | wc -l

    echo "novel tumor-specific transcripts NOT evidenced in miTranscriptome"
    cat data/$proj/analysis/mitranscriptome/nooverlap_mitranscriptome.gtf | cut -d";" -f2 | sort | uniq | wc -l
done